# Converter for the Financial Filings at the Sec

In [1]:
from zipfile import ZipFile
import pandas as pd

dirname = "2024q4"
with ZipFile('importFiles/'+dirname+'.zip') as myzip:
    with myzip.open('num.txt') as myfile1:
        dfNum = pd.read_table(myfile1,delimiter="\t", low_memory=False)
    with myzip.open('pre.txt') as myfile2:
        dfPre = pd.read_table(myfile2,delimiter="\t")
    with myzip.open('sub.txt') as myfile3:
        dfSub = pd.read_table(myfile3,delimiter="\t", low_memory=False)
    with myzip.open('tag.txt') as myfile4:
        dfTag = pd.read_table(myfile4,delimiter="\t")

dfSym = pd.read_table('importFiles/ticker.txt', delimiter="\t", header=None, names=['symbol','cik'])        

print(dfNum.size)
print(dfPre.size)
print(dfSub.size)
print(dfTag.size)
print(dfSym.size)

37050780
7375040
233676
759285
24168


In [8]:
dfNum.to_csv("exportFiles/num.csv", index=False)
dfPre.to_csv("exportFiles/pre.csv", index=False)
dfSub.to_csv("exportFiles/sub.csv", index=False)
dfTag.to_csv("exportFiles/tag.csv", index=False)

In [2]:
from tabulate import tabulate

print(tabulate(dfNum.head(100), headers='keys', tablefmt='psql'))
print(dfNum.dtypes)

+----+----------------------+--------------------------------------------------------------------------------------------------------------+----------------------+----------+--------+--------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+-------------------+------------+
|    | adsh                 | tag                                                                                                          | version              |    ddate |   qtrs | uom    | segments                                                                                                                                                                                                                                                                              | coreg   |  

In [3]:
print(tabulate(dfPre.head(100), headers='keys', tablefmt='psql'))
print(dfNum.dtypes)

+----+----------------------+----------+--------+--------+---------+---------+----------------------------------------------------------------------------------------------------------------+----------------------+----------------------------------------------------------------------------------------------------------------------+------------+
|    | adsh                 |   report |   line | stmt   |   inpth | rfile   | tag                                                                                                            | version              | plabel                                                                                                               |   negating |
|----+----------------------+----------+--------+--------+---------+---------+----------------------------------------------------------------------------------------------------------------+----------------------+--------------------------------------------------------------------------------------------

In [4]:
print(tabulate(dfSub.head(100), headers='keys', tablefmt='psql'))
print(dfNum.dtypes)

+----+----------------------+---------+------------------------------------------+-------+-------------+----------+----------------------+------------+------------------------------------------+--------------------------------+---------------------+-------------+----------+----------------------+------------+------------------------------------------+--------------------------------+--------------+-----------+-----------+--------------------------------------------+---------------+-------+--------+-------+--------+-------------+------+------+----------+-----------------------+-----------+----------+---------------------------------+---------+---------------+
|    | adsh                 |     cik | name                                     |   sic | countryba   | stprba   | cityba               | zipba      | bas1                                     | bas2                           | baph                | countryma   | stprma   | cityma               | zipma      | mas1                  

In [6]:
dfSub[dfSub['adsh'] == '0000950170-24-122438']

,adsh,cik,name,sic,countryba,stprba,cityba,zipba,bas1,bas2,...,period,fy,fp,filed,accepted,prevrpt,detail,instance,nciks,aciks
1,0000950170-24-122438,1720116,"RED VIOLET, INC.",7372.0,US,FL,BOCA RATON,33431,2650 NORTH MILITARY TRAIL,SUITE 300,...,20240930.0,2024.0,Q3,20241106,2024-11-06 16:55:00.0,0,1,rdvt-20240930_htm.xml,1,NaN


In [24]:
print(tabulate(dfTag.head(100), headers='keys', tablefmt='psql'))
print(dfNum.dtypes)

+----+----------------------------------------------------------------------------------------------------------------+--------------+----------+------------+------------+--------+--------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [25]:
print(tabulate(dfSym.head(10), headers='keys', tablefmt='psql'))
print(dfNum.dtypes)

+----+----------+---------+
|    | symbol   |     cik |
|----+----------+---------|
|  0 | aapl     |  320193 |
|  1 | msft     |  789019 |
|  2 | brk-b    | 1067983 |
|  3 | unh      |  731766 |
|  4 | jnj      |  200406 |
|  5 | v        | 1403161 |
|  6 | tsm      | 1046179 |
|  7 | xom      |   34088 |
|  8 | wmt      |  104169 |
|  9 | spy      |  884394 |
+----+----------+---------+
adsh         object
tag          object
version      object
ddate         int64
qtrs          int64
uom          object
segments     object
coreg        object
value       float64
footnote     object
dtype: object


In [26]:
print(dfNum.iloc[0])
print(dfNum.columns.values)

adsh                      0000950170-24-120982
tag                              NetIncomeLoss
version                           us-gaap/2024
ddate                                 20240331
qtrs                                         1
uom                                        USD
segments    EquityComponents=RetainedEarnings;
coreg                                      NaN
value                              -20331000.0
footnote                                   NaN
Name: 0, dtype: object
['adsh' 'tag' 'version' 'ddate' 'qtrs' 'uom' 'segments' 'coreg' 'value'
 'footnote']


In [27]:
from datetime import date, datetime, timedelta
from marshmallow import Schema, fields

class FinancialElementImportDto:
    label = ""          #Tag.doc
    concept = ""        #Num.tag
    info = ""           #Pre.plabel
    unit = ""           #Num.uom
    value = 0.0         #Num.value

class FinancialsDataDto:
    bs = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    cf = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    ic = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    

class SymbolFinancialsDto:
    startDate = date.today()   #Sub.period 
    endDate = date.today()     #Sub.period + Sub.fp
    year = 0                   #Sub.fy
    quarter = ""               #Sub.fp
    symbol = ""                #Sub.cik -> Sym.cik -> Sym.symbol
    name = ""                  #Sub.name
    country = ""               #Sub.countryma
    city = ""                  #Sub.cityma
    data = FinancialsDataDto()
    
class FinancialElementImportSchema(Schema):
    label = fields.String()
    concept = fields.String()
    info = fields.String()
    unit = fields.String()
    value = fields.Int()
    
class FinancialsDataSchema(Schema):
    bs = fields.List(fields.Nested(FinancialElementImportSchema()))
    cf = fields.List(fields.Nested(FinancialElementImportSchema()))
    ic = fields.List(fields.Nested(FinancialElementImportSchema()))
    
class SymbolFinancialsSchema(Schema):
    startDate = fields.DateTime()
    endDate = fields.DateTime()
    year = fields.Int()
    quarter = fields.String()
    symbol = fields.String()
    name = fields.String()
    country = fields.String()
    city = fields.String()
    data = fields.Nested(FinancialsDataSchema)

In [9]:
import os
# dirname = "output"
# if not os.path.exists("exportFiles/"+dirname):
#     os.makedirs("exportFiles/"+dirname)
isExist = os.path.exists("exportFiles/"+dirname)

# isExist = 1 == 2
if isExist:
    print("Reached here 1")
    for filename in os.listdir("exportFiles/"+dirname):
        with open(os.path.join("exportFiles/"+dirname, filename), 'r') as file:
            jsonStr = file.read()
            file.close()
        jsonStr = jsonStr.replace("'", '"').replace(", ",",").replace('\\r', '').replace('\\n', ' ')
        jsonStr = ''.join(jsonStr.splitlines())
        with open(os.path.join("exportFiles/"+dirname, filename), 'w') as file:    
            file.write(jsonStr)
            file.close()
    

Reached here 1


# Create a single SymbolFinancialsDto

In [34]:
#json_string = json.dumps([ob.__dict__ for ob in list_name])
#rslt_df = dataframe[dataframe['Percentage'] > 80]

import numpy as np
from pprint import pprint
import os
from dateutil.relativedelta import relativedelta
import json

def npInt_to_str(var):
    return str(list(np.reshape(np.asarray(var), (1, np.size(var)))[0]))[1:-1]

def formatDateNpNum(var):
    # dateStr = npInt_to_str(int(var))
    dateStr = str(var)
    return dateStr[0:4]+"-"+dateStr[4:6]+"-"+dateStr[6:8]

fileStartTime = datetime.now()
for subId in range(len(dfSub)):
    startTime = datetime.now()
    submitted = dfSub.iloc[subId]
    sfDto = SymbolFinancialsDto()
    sfDto.data = FinancialsDataDto()
    sfDto.data.bs = []
    sfDto.data.cf = []
    sfDto.data.ic = []
    try:
        period_value = int(submitted["period"])
        periodStartDate = date.fromisoformat(formatDateNpNum(period_value))
    except ValueError:
        print("File exportFiles/"+dirname+"/"+submitted["adsh"]+".json has Period: "+str(formatDateNpNum(period_value)), type(formatDateNpNum(period_value)))
        continue       
    sfDto.startDate = periodStartDate
    sfDto.endDate = date.today() #default value
    if submitted["fy"] == np.nan or np.isnan(submitted["fy"]):
        sfDto.year = 0
    else: 
        sfDto.year = submitted["fy"].astype(int)        
    sfDto.quarter = str(submitted["fp"]).strip().upper()
    
    if sfDto.quarter == "FY" or sfDto.quarter == "CY":
        sfDto.endDate = periodStartDate + relativedelta(months=+12, days=-1)
    elif sfDto.quarter == "H1" or sfDto.quarter == "H2":
        sfDto.endDate = periodStartDate + relativedelta(months=+6, days=-1)
    elif sfDto.quarter == "T1" or sfDto.quarter == "T2" or sfDto.quarter == "T3":
        sfDto.endDate = periodStartDate + relativedelta(months=+4, days=-1)
    elif sfDto.quarter == "Q1" or sfDto.quarter == "Q2" or sfDto.quarter == "Q3" or sfDto.quarter == "Q4":
        sfDto.endDate = periodStartDate + relativedelta(months=+3, days=-1)
    else:
        continue
        
    #symStr = dfSym[dfSym["cik"]==submitted.cik].symbol.str.upper().astype("string")
    #print(symStr)
    val = dfSym[dfSym["cik"]==submitted["cik"]]
    if len(val) > 0:
        sfDto.symbol = val["symbol"].to_string(index = False).split("\n")[0].split("\\n")[0].strip().split(" ")[0].strip().upper()
        if len(sfDto.symbol) > 19 or len(sfDto.symbol) < 1:
            print("File exportFiles/"+dirname+"/"+submitted["adsh"]+".json has Symbol "+sfDto.symbol)
    else:
        print("File exportFiles/"+dirname+"/"+submitted["adsh"]+".json has Symbol "+val["symbol"].to_string(index = False))
        continue
        
    sfDto.name = submitted["name"]
    sfDto.country = submitted["countryma"]
    sfDto.city = submitted["cityma"]

    sdDto = FinancialsDataDto()
    #print(filteredDfPre)
    dfNum['value'] = pd.to_numeric(dfNum['value'], errors='coerce')
    dfNum = dfNum.dropna(subset=['value'])
    dfNum['value'] = dfNum['value'].astype(int)
    #filteredDfNum = dfNum[dfNum['adsh'] == submitted['adsh']]       
    filteredDfNum = dfNum.loc[dfNum['adsh'].values == submitted['adsh']]
    filteredDfNum.reset_index()

    for myId in range(len(filteredDfNum)): 
        #print(myId)
        #print(filteredDfNum.iloc[myId])
        myNum = filteredDfNum.iloc[myId]
        myDto = FinancialElementImportDto()
        myTag = dfTag[dfTag["tag"] == myNum['tag']]
        myDto.label = myTag["doc"].to_string(index = False)
        myDto.concept = myNum["tag"]
        #myPre = dfPre[(dfPre['adsh'] == submitted["adsh"]) & (dfPre['tag'] == myNum['tag'])]
        myPre = dfPre.loc[np.where((dfPre['adsh'].values == submitted["adsh"]) & (dfPre['tag'].values == myNum['tag']))]
        myDto.info = myPre["plabel"].to_string(index = False).replace('"',"'")
        myDto.unit = myNum["uom"]
        #print(myNum["value"])        
        myDto.value = myNum["value"]
        #print(myDto.__dict__)
        #FinancialElementImportSchema().dump(myDto)
        if myPre['stmt'].to_string(index = False).strip() == 'BS':
            sfDto.data.bs.append(myDto)
        elif myPre['stmt'].to_string(index = False).strip() == 'CF':
            sfDto.data.cf.append(myDto)
        elif myPre['stmt'].to_string(index = False).strip() == 'IC':
            sfDto.data.ic.append(myDto)

    #print(len(filteredDfNum))
    #print(submitted)
    #print(sfDto.__dict__)    
    #json.dumps(sfDto.__dict__)    
    marshmellowStart = datetime.now()
    result = SymbolFinancialsSchema().dump(sfDto)
    result = str(json.dumps(result))
    #result = result.replace('"','öäü').replace("'", '"').replace("öäü","'").replace(", ",",")
    result = result.replace('\\r', '').replace('\\n', ' ')
    
    print("FilterDfNum size: "+str(len(filteredDfNum))+" Dtos size: "+str((len(sfDto.data.bs)+len(sfDto.data.cf)+len(sfDto.data.ic)))+" Bs: "+str(len(sfDto.data.bs))+" Cf: "+str(len(sfDto.data.cf))+" Ic: "+str(len(sfDto.data.ic))+" Json size: "+str(int(len(str(result)) / 1024))+"kb.")
    
    #print("Json size "+str(int(len(str(result)) / 1024))+"kb. Time: "+str((datetime.now() - marshmellowStart) / timedelta(milliseconds=1))+"ms.")
    
    #print(result)
    #pprint(result)    
    isExist = os.path.exists("exportFiles/"+dirname)
    if not isExist:
        os.makedirs("exportFiles/"+dirname)
    json_file = open("exportFiles/"+dirname+"/"+submitted["adsh"]+".json","w")
    json_file.write(result)
    json_file.close()
    endTime = datetime.now()
    print("file "+str(subId+1)+" of "+str(len(dfSub)+1)+" exportFiles/"+dirname+"/"+submitted["adsh"]+".json stored in "+str((endTime - startTime) / timedelta(milliseconds=1))+"ms.")
    
fileEndTime = datetime.now()
print("Time "+str((fileEndTime - fileStartTime) / timedelta(seconds=1))+"sec for "+str(len(dfSub)+1)+" files.")

FilterDfNum size: 516 Dtos size: 186 Bs: 106 Cf: 80 Ic: 0 Json size: 45kb.
file 1 of 6492 exportFiles/2024q4/0001628280-24-043777.json stored in 29512.162ms.
FilterDfNum size: 282 Dtos size: 125 Bs: 71 Cf: 54 Ic: 0 Json size: 46kb.
file 2 of 6492 exportFiles/2024q4/0000950170-24-122438.json stored in 15892.353ms.
FilterDfNum size: 461 Dtos size: 162 Bs: 108 Cf: 54 Ic: 0 Json size: 38kb.
file 3 of 6492 exportFiles/2024q4/0000060667-24-000169.json stored in 26342.782ms.
File exportFiles/2024q4/0002002473-24-000015.json has Symbol Series([], )
FilterDfNum size: 432 Dtos size: 171 Bs: 103 Cf: 68 Ic: 0 Json size: 41kb.
file 5 of 6492 exportFiles/2024q4/0000950170-24-123804.json stored in 26508.306ms.
FilterDfNum size: 154 Dtos size: 87 Bs: 62 Cf: 25 Ic: 0 Json size: 20kb.
file 6 of 6492 exportFiles/2024q4/0001477932-24-006462.json stored in 9582.335ms.
FilterDfNum size: 244 Dtos size: 122 Bs: 85 Cf: 37 Ic: 0 Json size: 28kb.
file 7 of 6492 exportFiles/2024q4/0001213900-24-086156.json stored

KeyboardInterrupt: 